In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

delta_package = "io.delta:delta-spark_2.12:3.0.0"  # Replace with the correct Delta version
xml_package = "com.databricks:spark-xml_2.12:0.14.0"
# Initialize Spark Session
#spark = SparkSession.builder.appName("MergeToProcessed").master('spark://spark-test1:7077') \
spark = SparkSession.builder.appName("MergeToProcessed") \
    .config("spark.jars.packages", f"{delta_package},{xml_package}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.cores.max", "1") \
    .getOrCreate()

hdfs_path = "hdfs://spark-test1:9000"
raw = os.path.join(hdfs_path, 'raw', 'transactions')
checkpoint = os.path.join(hdfs_path, 'checkpoint', 'processed', 'transactions')
dlq = os.path.join(hdfs_path, 'dlq', 'processed', 'transactions')


:: loading settings :: url = jar:file:/home/spark/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2e62ddf1-96d9-4193-aab5-64ff359dcf20;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 233ms :: artifacts dl 40ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [def

In [2]:
from pyspark.sql.column import Column, _to_java_column
from pyspark.sql.types import _parse_datatype_json_string

def ext_from_xml(xml_column, schema, options={}):
    java_column = _to_java_column(xml_column.cast('string'))
    java_schema = spark._jsparkSession.parseDataType(schema.json())
    scala_map = spark._jvm.org.apache.spark.api.python.PythonUtils.toScalaMap(options)
    jc = spark._jvm.com.databricks.spark.xml.functions.from_xml(
        java_column, java_schema, scala_map)
    return Column(jc)

def ext_schema_of_xml_df(df, options={}):
    assert len(df.columns) == 1

    scala_options = spark._jvm.PythonUtils.toScalaMap(options)
    java_xml_module = getattr(getattr(
        spark._jvm.com.databricks.spark.xml, "package$"), "MODULE$")
    java_schema = java_xml_module.schema_of_xml_df(df._jdf, scala_options)
    return _parse_datatype_json_string(java_schema.json())

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

# Define the schema of your raw layer
raw_schema = StructType([
    StructField("key", StringType()),
    StructField("value", StringType()),
    StructField("topic", StringType()),
    StructField("partition", StringType()),
    StructField("offset", StringType()),
    StructField("timestamp", StringType()),
    StructField("timestampType", StringType()),
    StructField("_etl_insert_timestamp", TimestampType()),
    StructField("_etl_insert_date", StringType()),
    StructField("_etl_insert_hour", IntegerType())
])
# Read the data from the raw layer using structured streaming
raw_df = spark.readStream.format("parquet")\
    .option("path", raw)\
    .option("maxFilesPerTrigger", 1)\
    .schema(raw_schema) \
    .load()

json_schema = StructType([
    StructField("path", StringType()),
    StructField("modificationTime", StringType()),
    StructField("length", StringType()),
    StructField("content", StringType())
])
parsed_json_df = raw_df.withColumn("json_data", from_json(col("value"), json_schema))

# Extract and decode the base64 content
decoded_df = parsed_json_df.withColumn("decoded_content", unbase64(col("json_data.content"))) \
    .withColumn("xml_content", expr("CAST(decoded_content AS STRING)"))

#schema_def = ext_schema_of_xml_df(decoded_df.select("xml_content"))
#decoded_df = decoded_df.withColumn('test_debug', lit(schema_def).cast('string'))

xml_schema = StructType([
    StructField(
        'Transaction', 
        ArrayType(
            StructType([
                StructField('TransactionId', LongType(), True),
                StructField('Amount', FloatType(), True),
                StructField('CustomerId', LongType(), True),
                StructField('DateTime', TimestampType(), True),
                StructField('Location', StringType(), True),
                StructField('Result', StringType(), True)
            ]),
            True
        ),
        True
    )
])

xml_df = decoded_df.withColumn(
    "parsed",
    ext_from_xml(
        xml_column = col("xml_content"),
        schema=xml_schema,
        options={"mode": "FAILFAST"}
    )
)
# Flatten the DataFrame
flattened_df = xml_df.select(
    explode(col("parsed.Transaction")).alias("Transaction")
)
flattened_df.printSchema()
flattened_df = flattened_df.select(
    col("Transaction.Amount").alias("Amount"),
    col("Transaction.CustomerId").alias("CustomerId"),
    col("Transaction.DateTime").alias("DateTime"),
    upper(trim(col("Transaction.Location"))).alias("Location"),
    upper(trim(col("Transaction.Result"))).alias("Result"),
    current_date().alias("_etl_insert_date"),
    date_format(current_timestamp(), "HH").alias("_etl_insert_hour"),
    current_timestamp().alias("_etl_insert_timestamp")
)

# Write the transformed data to the processed layer
query = flattened_df.writeStream\
    .outputMode("append")\
    .option("checkpointLocation", checkpoint)\
    .trigger(once=True)\
    .format("parquet")\
    .option("path", processed_layer_path)\
    .start()

# query = flattened_df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()

root
 |-- Transaction: struct (nullable = true)
 |    |-- TransactionId: long (nullable = true)
 |    |-- Amount: float (nullable = true)
 |    |-- CustomerId: long (nullable = true)
 |    |-- DateTime: timestamp (nullable = true)
 |    |-- Location: string (nullable = true)
 |    |-- Result: string (nullable = true)



23/12/10 05:13:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9c8ecb87-9b11-4652-9b61-dae6bf685e28. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/10 05:13:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
